In [1]:
import pandas as pd
from pathlib import Path
import requests
from tqdm import tqdm

In [73]:
DATA_DIR = Path("data")

all_symbols = pd.read_csv(DATA_DIR / "all_symbols.csv").rename(columns={"0": "symbol"})

In [74]:
all_symbols

,symbol
0,MRX
1,JOB
2,WALLET
3,MSB
4,XCHNG
...,...
1422,MCB
1423,HT
1424,AE
1425,DIONE


In [100]:
import requests
import pandas as pd


def query_subgraph(url, token_symbol):
    query = f"""
    {{
        tokens(where: {{symbol: "{token_symbol}"}}) {{
            id
            symbol
            name
            totalValueLockedUSD
            whitelistPools {{
                id
                token0 {{
                    id
                    symbol
                    name
                }}
                token1 {{
                    id
                    symbol
                    name
                }}
            volumeUSD
            }}
        }}
    }}
    """
    response = requests.post(url, json={"query": query})
    if response.status_code == 200:
        return response.json()
    else:
        return None


# List of token symbols to search
token_symbols = [
    "USDC",
    "WBTC",
    "ETH",
    "WETH",
    "wTAO",
] + all_symbols[
    "symbol"
].tolist()  # [:200]

subgraphs = {
    "Ethereum Mainnet": "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",
    # 'Polygon': "https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-polygon",
    #'Optimism': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3-optimism"
    # Add more chains as needed
}

results = []
for token_symbol in tqdm(token_symbols):
    for chain, url in subgraphs.items():
        data = query_subgraph(url, token_symbol)
        if data and data.get("data", {}).get("tokens"):
            tokens = data["data"]["tokens"]
            for token in tokens:
                for pool in token["whitelistPools"]:
                    results.append(
                        {
                            "Chain": chain,
                            "Token Symbol": token["symbol"],
                            "Token Name": token["name"],
                            "Token ID": token["id"],
                            "Pool ID": pool["id"],
                            "Token0 ID": pool["token0"]["id"],
                            "Token0 Symbol": pool["token0"]["symbol"],
                            "Token0 Name": pool["token0"]["name"],
                            "Token1 ID": pool["token1"]["id"],
                            "Token1 Symbol": pool["token1"]["symbol"],
                            "Token1 Name": pool["token1"]["name"],
                            "Volume USD": float(pool["volumeUSD"]),
                            "TVL USD": float(token["totalValueLockedUSD"]),
                        }
                    )

df = pd.DataFrame(results)
df = df.drop_duplicates(subset=["Pool ID"])

# remove 0 tvl adn volume
df = df[df["TVL USD"] > 0]
df = df[df["Volume USD"] > 0]

df["Pool Name"] = df["Token0 Symbol"] + "/" + df["Token1 Symbol"]

# Optionally, save the DataFrame to a CSV file
# df.to_csv('uniswap_pools.csv', index=False)

 22%|██▏       | 319/1432 [01:56<06:57,  2.66it/s]

In [ ]:
df

,Chain,Token Symbol,Token Name,Token ID,Pool ID,Token0 ID,Token0 Symbol,Token0 Name,Token1 ID,Token1 Symbol,Token1 Name,Volume USD,TVL USD
3,Ethereum Mainnet,USDC,USD Coin (PoS) (Wormhole),0x566957ef80f9fd5526cd2bef8be67035c0b81130,0xd376814cd28dcb6f30553e5026708ec9d6b9a8dc,0x566957ef80f9fd5526cd2bef8be67035c0b81130,USDC,USD Coin (PoS) (Wormhole),0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,3.715274e+06,1.855657e+02
8,Ethereum Mainnet,USDC,UltraSuperfuckingDementedgreendildoretardstren...,0x9abc68b33961268a3ea4116214d7039226de01e1,0x07a72f8f6a29cf501e7226ca82264f9ee79380e7,0x9abc68b33961268a3ea4116214d7039226de01e1,USDC,UltraSuperfuckingDementedgreendildoretardstren...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,1.750676e+07,3.195620e+05
9,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x020c349a0541d76c16f501abc6b2e9c98adae892,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,SNX,Synthetix Network Token,1.121544e+08,6.130743e+08
10,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x026babd2ae9379525030fc2574e39bc156c10583,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,3.276979e+05,6.130743e+08
11,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x07a6e955ba4345bae83ac2a6faa771fddd8a2011,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Matic Token,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,1.196317e+08,6.130743e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0x56027930bdff39faf295a7cf6bb94b9bb9f6a718,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,2.349600e+06,2.495186e+06
525,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0xad25a15d9059a8d3163acbbe6a98119fc63afbaf,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,4.912814e+01,2.495186e+06
526,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0xcd83055557536eff25fd0eafbc56e74a1b4260b3,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,8.258626e+08,2.495186e+06
527,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0xe9ac65ca67cf1cd5124d030bbbcb2f9f2b1cf72e,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Tether USD,1.999999e+00,2.495186e+06


In [98]:
df.sort_values("TVL USD", ascending=False).groupby("Token Symbol").head(2)

,Chain,Token Symbol,Token Name,Token ID,Pool ID,Token0 ID,Token0 Symbol,Token0 Name,Token1 ID,Token1 Symbol,Token1 Name,Volume USD,TVL USD
128,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x60594a405d53811d3bc4766596efd80fd545a270,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,Dai Stablecoin,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,3.025558e+10,2.115144e+09
111,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x04916039b1f59d9745bf6e0a21f191d1e0a84287,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,YFI,yearn.finance,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,3.176280e+08,2.115144e+09
44,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xbb256c2f1b677e27118b0345fd2b3894d2e6d487,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Tether USD,2.214721e+05,6.130743e+08
32,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,4.730828e+11,6.130743e+08
64,Ethereum Mainnet,WBTC,Wrapped BTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,6.782049e+10,3.589768e+08
60,Ethereum Mainnet,WBTC,Wrapped BTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x15c11b1e07cb763ab75723bb55877b36f56c39b1,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,SNX,Synthetix Network Token,4.374002e+05,3.589768e+08
473,Ethereum Mainnet,LINK,ChainLink Token,0x514910771af9ca656af840dff83e8264ecf986ca,0xd92ba9b67b74305b5534abd2b1f5119b59b24871,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,YFI,yearn.finance,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,ChainLink Token,1.030134e+03,3.489161e+07
471,Ethereum Mainnet,LINK,ChainLink Token,0x514910771af9ca656af840dff83e8264ecf986ca,0xcfc389e23728f72558eee13d04b326ddcdd88be3,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,ChainLink Token,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,Matic Token,1.607353e+04,3.489161e+07
523,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0x1eefc75cc4458e651480e80d74b263b77a93cb11,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,3.685471e+07,2.495186e+06
524,Ethereum Mainnet,PERP,Perpetual,0xbc396689893d065f41bc2c6ecbee5e0085233447,0x56027930bdff39faf295a7cf6bb94b9bb9f6a718,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xbc396689893d065f41bc2c6ecbee5e0085233447,PERP,Perpetual,2.349600e+06,2.495186e+06


In [87]:
for name, group in df.groupby("Token0 Symbol"):
    print(name)
    # print address
    print(group["Token0 ID"].values[0])

    # print pair

1INCH
0x111111111117dc0aa78b770fa6a738034120c302
1INCH Token
613074273.41451
AAVE
0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
Aave Token
613074273.41451
AGLD
0x32353a6c91143bfd6c7d363b546e62a9a2489a20
Adventure Gold
1986804.903946648
AKRO
0x8ab7404063ec4dbcfd4598215992dc3f8ec853d7
Akropolis
0.0
ALICE
0xac51066d7bec65dc4589368da368b212745d63e8
ALICE
0.0
ALL
0x913749155f1378fa482f5d2889fde3ada3b7515c
OrdinallDAO
0.0
ALPHA
0x146d0fff5be0c7825f47ca0359c3070b12a4684b
Alpha
0.0
AMPL
0xd46ba6d942050d489dbd938a2c909a5d5039a161
Ampleforth
0.0
APL
0x60f886c6fd97aa59e9e1e81cf5f51c4a25c6cd2d
Approximate Labs
0.0
ASM
0x2565ae0385659badcada1031db704442e1b69982
ASSEMBLE
0.0
AVA
0xa6c0c097741d55ecd9a3a7def3a8253fd022ceb9
AVA
0.0
AVT
0x0d88ed6e74bbfd96b831231638b66c05571e824f

0.0
BCUG
0x14da7b27b2e0fedefe0a664118b0c9bc68e2e9af
Blockchain Cuties Universe Governance Token
0.0
BDX
0x709e46ce1eb677c3beccc32a27a2e71eaaf7a7be
BODAX
0.0
BEND
0x0d02755a5700414b26ff040e1de35d337df56218
Bend Token
0.0
BGB
0x6731

In [ ]:
selected_pools_df = (
    df.sort_values("Volume USD", ascending=False)
    .groupby("pair_name")
    .first()
    .sort_values("Volume USD", ascending=False)
)

print(len(selected_pools_df))
selected_pools_df

66


,Chain,Token Symbol,Token Name,Token ID,Pool ID,Token0 ID,Token0 Symbol,Token0 Name,Token1 ID,Token1 Symbol,Token1 Name,Volume USD,TVL USD
pair_name,,,,,,,,,,,,,
USDC/WETH,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,4.730827e+11,6.130151e+08
WETH/USDT,Ethereum Mainnet,USDT,Tether USD,0xdac17f958d2ee523a2206206994597c13d831ec7,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Tether USD,9.516109e+10,2.500036e+08
USDC/USDT,Ethereum Mainnet,USDC,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3416cf6c708da44db2624d63ea0aaef7113527c6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD Coin,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Tether USD,8.027196e+10,6.130151e+08
WBTC/WETH,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,6.782041e+10,2.115345e+09
DAI/WETH,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x60594a405d53811d3bc4766596efd80fd545a270,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,Dai Stablecoin,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,3.025558e+10,2.115345e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
sUSD/WETH,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x6e2a6e92d1c0df4fbf7b35e9aebf2e681c9e6f5f,0x57ab1ec28d129707052df4df418d58a2d46d5f51,sUSD,Synth sUSD,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,5.563937e+04,2.115345e+09
ETH/WETH,Ethereum Mainnet,ETH,ETH,0x5884658a803a3655e8955b202cad9956339649dc,0x1974f23344bf728fc5e0d5a04ca8e2b0a6533de4,0x5884658a803a3655e8955b202cad9956339649dc,ETH,ETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,4.740474e+04,1.006896e-13
cDAI/WETH,Ethereum Mainnet,WETH,Wrapped Ether,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x82bd0e16516f828a0616038002e152aa6f27aedc,0x5d3a536e4d6dbd6114cc1ead35777bab948e3643,cDAI,Compound Dai,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,9.906405e+03,2.115345e+09
